In [2]:
#uselful snorkel sources:
#https://www.snorkel.org/use-cases/01-spam-tutorial#2-writing-labeling-functions-lfs 3a) keyword lookup
#https://www.snorkel.org/use-cases/01-spam-tutorial#2-writing-labeling-functions-lfs 3e) preprocessor


## Setup

# Imports

In [2]:
import numpy as np
import pandas as pd
import os
import tqdm
import pickle
from tqdm import tqdm

from functools import lru_cache

from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

import warnings

## Datasets

In [3]:
from load_datasets import load_asset_ds
from load_datasets import load_automets_ds
from load_datasets import load_benchls_ds
from load_datasets import load_britannica_ds
from load_datasets import load_dwikipedia_ds
from load_datasets import load_ewsewgmpm_ds
from load_datasets import load_ewsewturk_ds
from load_datasets import load_htss_ds
from load_datasets import load_hutssf_ds
from load_datasets import load_massalign_ds
from load_datasets import load_metaeval_ds
from load_datasets import load_mturksf_ds
from load_datasets import load_nnseval_ds
from load_datasets import load_onestopenglish_ds
from load_datasets import load_pwkp_ds
from load_datasets import load_semeval07_ds
from load_datasets import load_simpa_ds
from load_datasets import load_simpeval_ds
from load_datasets import load_sscorpus_ds
from load_datasets import load_turkcorpus_ds
from load_datasets import load_wikiauto_ds
from load_datasets import load_wikimanual_ds
from load_datasets import load_wikisplit_ds
from load_datasets import load_wikipediav1_ds
from load_datasets import load_wikipediav2_ds
from load_datasets import path_to_datasets

if not os.path.isdir(path_to_datasets):
    os.mkdir(path_to_datasets)

asset = load_asset_ds()
automets = load_automets_ds()
benchls = load_benchls_ds()
britannica = load_britannica_ds()
dwikipedia = load_dwikipedia_ds()
ewsewgmpm = load_ewsewgmpm_ds()
ewsewturk = load_ewsewturk_ds()
htss = load_htss_ds()
hutssf = load_hutssf_ds()
massalign = load_massalign_ds()
metaeval = load_metaeval_ds()
mturksf = load_mturksf_ds()
nnseval = load_nnseval_ds()
onestopenglish = load_onestopenglish_ds()
pwkp = load_pwkp_ds()
semeval07 = load_semeval07_ds()
simpa = load_simpa_ds()
simpeval = load_simpeval_ds()
sscorpus = load_sscorpus_ds()
turkcorpus = load_turkcorpus_ds()
wikiauto = load_wikiauto_ds()
wikimanual = load_wikimanual_ds()
wikisplit = load_wikisplit_ds()
wikipediav1 = load_wikipediav1_ds()
wikipediav2 = load_wikipediav2_ds()

combined_dataset = pd.concat([asset, automets, benchls, britannica, dwikipedia, ewsewgmpm, ewsewturk, htss, hutssf, massalign, 
                              metaeval, mturksf, nnseval, onestopenglish, pwkp, semeval07, simpa, simpeval, sscorpus, turkcorpus, 
                              wikiauto, wikimanual, wikisplit, wikipediav1, wikipediav2], axis=0).reset_index()

with open('/' + path_to_datasets + '/combined_dataset.pkl', 'wb') as f:
    pickle.dump(combined_dataset, f)

Cloning into 'wiki-auto'...
Updating files: 100% (248/248), done.


In [10]:
metadata_ds = pd.read_excel('/workspace/datasets/English_Datasets.xlsx')

merged_ds = pd.merge(metadata_ds[['ds_id', 'Year', 'Target_Audience', 'Domain']], combined_dataset, on=['ds_id'])

with open('/workspace/datasets/final_combined.pkl', 'wb') as f:
    pickle.dump(merged_ds, f)

## Meaning Preservation

In [3]:
with open("/workspace/datasets/combined_dataset.pkl", 'rb') as f:
    combined_dataset = pickle.load(f)

In [4]:
from labeling_functions import get_all_lfs

2023-10-10 13:37:29.753115: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 13:37:29.789584: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 13:37:29.790121: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 13:37:30.569481: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
ABSTAIN = -1
SIMPLE = 0
NOT_SIMPLE = 1
LOST_MEANING = 2

label_map = {-1: "ABSTAIN", 0: "SIMPLE", 1: "NOT_SIMPLE", 2: "LOST_MEANING"}

In [ ]:
all_lfs = get_all_lfs()

applier = PandasLFApplier(all_lfs)
with warnings.catch_warnings():
  warnings.simplefilter("ignore")
  labels = applier.apply(sub_sample)

In [ ]:
#show some stats for the results
from snorkel.labeling import LFAnalysis
LFAnalysis(L=labels, lfs=all_lfs).lf_summary()

In [ ]:
test_l = labels[:,:250]

In [ ]:
test_l.shape

In [ ]:
#apply snorkel magic and automatically combine labels
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=labels, n_epochs=500, log_freq=5, seed=42, lr=0.001)

In [ ]:
#snorkel generates noisy labels based on the aggregation of signals provided by the labeling functions
label_model_preds = label_model.predict(L=labels)
label_model_pred_probs = label_model.predict_proba(L=labels)

In [ ]:
#explain label model
weights = label_model.get_weights()

for i in range(len(all_lfs)):
  print(f"{all_lfs[i].name} : {weights[i]}")

In [ ]:
def check_instance(id):
  print(f"src_snt : {sub_sample.iloc[id]['source_snt']}")
  print(f"simp_snt : {sub_sample.iloc[id]['simplified_snt']}")
  print()
  print("Signals:")

  for i in range(len(all_lfs)):
    print(f"{all_lfs[i].name} : {label_map[labels[i][id]]}")
  print()
  print(f"complexity_score: {label_map[label_model_preds[id]]} ({label_model_pred_probs[id]})")
  print(f"gold label : {label_map[sub_sample.iloc[id]['gold_label']]}")

In [ ]:
res = label_model.score(L=labels, Y=sub_sample['gold_label'], metrics=['accuracy', 'f1'], tie_break_policy="random")

print(f"{'Label Model Accuracy:':<25} {res['accuracy'] * 100:.1f}%")
print(f"{'Label Model F1-Score:':<25} {res['f1'] * 100:.1f}%")

In [ ]:
res

In [ ]:
LFAnalysis(labels, all_lfs).lf_summary(sub_sample['gold_label'].values)

In [ ]:
for i in range(1):
  check_instance(i)
  print('-'*30)